<a href="https://colab.research.google.com/github/LeonSilva15/fcc-machine-learning/blob/main/sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import libraries

In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

## Get data files

In [ ]:
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
df_train = pd.read_csv(train_file_path, sep='\t', header=None, names=['y', 'x'])
df_train.head()

In [ ]:
df_test = pd.read_csv(test_file_path, sep='\t', header=None, names=['y', 'x'])
df_test.head()

## Transform categorical values

In [ ]:
y_train = df_train['y'].astype('category').cat.codes
y_test = df_test['y'].astype('category').cat.codes

In [ ]:
y_train.head()

In [ ]:
y_test.head()

In [ ]:
bar = df_train['y'].value_counts()

plt.bar(bar.index, bar)
plt.xlabel('label')
plt.title('Number of ham and spam messages')

## Text preprocessing

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [ ]:
stopwords_eng = set(stopwords.words('english'))
len(stopwords_eng)

In [ ]:
lemmatizer = WordNetLemmatizer()
def clean_text(text):
  txt = re.sub(r'([^\s\w])+', ' ', text)
  txt = ' '.join([lemmatizer.lemmatize(word) for word in txt.split()
                  if not word in stopwords_eng])
  txt = txt.lower()
  return txt

In [ ]:
X_train = df_train['x'].apply(lambda x: clean_text(x))
X_train.head()

## Vectorize

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

In [ ]:
# Max 1000 frequently ocurring words
max_words = 1000

# Max len 500 words
max_len = 500

In [ ]:
t = Tokenizer(num_words=max_words)
t.fit_on_texts(X_train)

In [ ]:
# Transform each text to a sequence of integers
sequences = t.texts_to_sequences(X_train)
sequences[:5]

In [ ]:
# Standardize the rows length
sequences_matrix = sequence.pad_sequences(sequences, maxlen=max_len)
sequences_matrix[:5]

## Create the model

In [ ]:
input = keras.layers.Input(shape=[max_len])
x = keras.layers.Embedding(max_words, 50)(input)
x = keras.layers.LSTM(64)(x)
x = keras.layers.Dense(128, activation='relu')(x)
x = keras.layers.Dense(256, activation='relu')(x)
x = keras.layers.Dense(128, activation='relu')(x)
x = keras.layers.Dropout(0.5)(x)
x = keras.layers.Dense(1, activation='relu')(x)

model = keras.models.Model(
    inputs=input,
    outputs=x
)

model.compile(
    loss='binary_crossentropy',
    optimizer='RMSprop',
    metrics=['accuracy']
)
model.summary()

In [ ]:
r = model.fit(sequences_matrix, y_train,
              batch_size=128, epochs=10,
              validation_split=0.2,
              callbacks=[keras.callbacks.EarlyStopping(
                  monitor='val_loss', min_delta=0.0001
              )])

## Evaluate

In [ ]:
plt.plot(r.history['loss'], label='Loss')
plt.plot(r.history['val_loss'], label='Validation loss')
plt.legend()

In [ ]:
plt.plot(r.history['accuracy'], label='Accuracy')
plt.plot(r.history['val_accuracy'], label='Validation accuracy')
plt.legend()

In [ ]:
def preprocess(X):
  x = X.apply(lambda x: clean_text(x))
  x = t.texts_to_sequences(X)
  return sequence.pad_sequences(x, maxlen=max_len)

In [ ]:
eval = model.evaluate(preprocess(df_test['x']), y_test)

In [ ]:
print('Loss: {:.3f}'.format(eval[0]))
print('Accuracy: {:.3f}'.format(eval[1]))

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  pred = model.predict(preprocess(pd.Series(pred_text)))[0]
  label = 'ham' if pred < 0.5 else 'spam'
  prediction = (pred[0], label)

  print(prediction)
  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
